<a href="https://colab.research.google.com/github/dakshchanchlani/Fake-news-detection/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [ ]:
import numpy as np #numpy is used for making numpy arrays


import pandas as pd #used for creating a data frame and storing the data in the dataframe

import re #regular expression(re) it is used for searching text in document  

from nltk.corpus import stopwords #natural language tool kit(nltk)   corpus is collection of text documents

# stopwards means words which doesn't add much value to the text or para

from nltk.stem.porter import PorterStemmer #(stemming takes the word and removes suffix and prefix of the word and take the root of the word)

from sklearn.feature_extraction.text import TfidfVectorizer #(it is used to convert the text into feature vectors(numbers))

from sklearn.model_selection import train_test_split #(used to split data into test and train datasets)

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score 

In [ ]:
import nltk

nltk.download('stopwords')

#downloded stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

#its printing all the common words from english 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing


In [ ]:
#laoding the dataset to a pandas dataset
news_dataset = pd.read_csv('train.csv', engine='python', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 11827: unexpected end of data


In [ ]:
news_dataset.shape    #to check total number of rows and columns in data



(11825, 5)

In [ ]:
#print the first 5 rows of the dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:

# counting the numbers of missing values in the dataset
news_dataset.isnull().sum()   #sum for totalling the number 

id           0
title      322
author    1101
text        25
label        0
dtype: int64

In [ ]:
#replacing null values with empty string just cuz we don't have lot number of missing values
news_dataset=news_dataset.fillna('')   #fillna means filling null string in missing values place

In [ ]:
#merging the author name and  news tile

news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

# here we have made new column name content and there we have stored the values of both author and title by writing above command

In [ ]:
print(news_dataset['content'])  #here we have printed new column content to check the changes

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
16997    Charlie Spiering Donald Trump and Family Atten...
16998    Julie Hirschfeld Davis Trump and Obama Hold Co...
16999    Tyler Durden Assange predicts Trump will lose,...
17000    Tom Ciccotta Gr-r-reat? Why Amazon’s Whole Foo...
17001    Somini Sengupta United Nations Chief Exposes L...
Name: content, Length: 17002, dtype: object


In [ ]:
# seperating the label from data
# column = (axis =1)
# row = (axis=0)

X= news_dataset.drop(columns='label',axis =1)

# here we dropped the label column from data and now X only contains all the columns except label

Y= news_dataset['label']
# we stored label column in Y so that we can call it later


In [ ]:
print(X)  # you will see all the columns except label
print(Y)  # in this you will just see label; column

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
11820  11820  ...  Dave Hodges Texas Elector Expects Massive Corr...
11821  11821  ...  Norm Bimstein (UID 71489130) Neocon Trotskyite...
11822  11822  ...  Michelle Oxman No Search Warrant Yet: FBI Can’...
11823  11823  ...  Kit O'Connell #NoDAPL Spills Over: Musicians B...
11824  11824  ...  Robert Fisk A View of the Syrian War From the ...

[11825 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
11820    1
11821    1
11822    1
11823    1
11824    1
Name: label, Length: 1182

Stemming is the process of reducing a word to its root words

Example:

actor, actoress, acting, -----> act

In [ ]:

port_stem = PorterStemmer()
#here we have created a shortcut for function by storing it in a variable


In [ ]:
def stemming(content):   

  stemmed_content = re.sub('[^a-zA-Z]',' ', content) 
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

1) We are creating a function called stemming and input as content(its not that we need to mention the same word as name of column we just did like this only)

2) re is being used for searching a paragrah for text
   sub is used for substituting the values

3) replace all the characters except a-z and A-Z with the empty string

4) split : to convert all the words into a list by using spacing and spliting

5) using a for loop we are iterating and we are doing stemming only for the words which are not stopwords

6) join all the stemmed content to make a paragraph


In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)
#here we are applying stemming to our content

In [ ]:
print(news_dataset['content'])
#printing the stemmed content

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
11820    dave hodg texa elector expect massiv corrupt r...
11821    norm bimstein uid neocon trotskyit hanniti wan...
11822    michel oxman search warrant yet fbi move email...
11823    kit connel nodapl spill musician boycott dakot...
11824             robert fisk view syrian war golan height
Name: content, Length: 11825, dtype: object


In [ ]:
#sepaerating the data and label
#WE ARE JUST MAKING THE LIST OF STEMMED DATA AND THERE ALSO WE ARE SEPERATING 
X = news_dataset['content'].values
print(X)
Y=news_dataset['label'].values
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michel oxman search warrant yet fbi move email investig'
 'kit connel nodapl spill musician boycott dakota access pipelin ceo record label festiv'
 'robert fisk view syrian war golan height']
[1 0 1 ... 1 1 1]


In [ ]:
Y.shape

(11825,)

In [ ]:
# converting the textual data into numerical daya

#tf is term frequency and idf is Inverse document frequency

#Tf : to count number of times a word is repeating in the particular text

#Tf assign the numerical value to the word which is repeating several times considered as important 
#idf : it assign the numerical values to the words which are repeating but having no significance 

vectorizer = TfidfVectorizer() #created a shortcut for TfidVectorizer function
vectorizer.fit(X)  #fitted all the data of X invectorizer =  vectorizer.transform(X

X = vectorizer.transform(X)  #converts all the words are convert into feature vectors


In [ ]:
print(X)

  (0, 12219)	0.28621144217362343
  (0, 10478)	0.2629798164626514
  (0, 6952)	0.35768376892660153
  (0, 6735)	0.2894056162896596
  (0, 6017)	0.24630927225626897
  (0, 5489)	0.22519583287943606
  (0, 3908)	0.23751243049296497
  (0, 2989)	0.27083817205627986
  (0, 2836)	0.35768376892660153
  (0, 2330)	0.24882075695474679
  (0, 1957)	0.36343918910076006
  (0, 225)	0.27229938406504056
  (1, 13040)	0.29517324650019117
  (1, 5353)	0.1883358222127907
  (1, 4333)	0.7189253422808268
  (1, 2811)	0.265412862886651
  (1, 2211)	0.18958711487573737
  (1, 1755)	0.3841018167959311
  (1, 1506)	0.1518827718628741
  (1, 1198)	0.28906553042707556
  (2, 12165)	0.42325392961598707
  (2, 7486)	0.48876787572256536
  (2, 4688)	0.3465537234196917
  (2, 4246)	0.3848045076337543
  (2, 2447)	0.45938315338966046
  :	:
  (11822, 7749)	0.2981663549995988
  (11822, 7460)	0.29882673753604944
  (11822, 5893)	0.26852505805379523
  (11822, 4126)	0.24477747352433543
  (11822, 3675)	0.24821927939570962
  (11823, 11136)	0.296

Splittig the dataset to training & test data 

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=3)

divided the content data into two parts 

X_train : training 
 
X_test : testing data

for getting 20% test data and training data 80% thats why test_size=0.2

Similarly for label means Y 
Y_train, Y_test

Stratify will proportionate X and Y equally basically the label was having the proportion of 1 and 0 so to mantain that we did stratify so that it assign (the same true or false to same news)


random state is just a way of splitting so to do it same as youtuber we are choosing 2 only

Training the Model : Logistic Regression


In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

*EVALUATION*

ACCURACY SCORE


In [ ]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("Accuracy Score of the Training data : ",training_data_accuracy)

Accuracy Score of the Training data :  0.9817124735729387


In [ ]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print("Accuracy Score of the Test data : ", test_data_accuracy)

Accuracy Score of the Test data :  0.9653276955602537


Making a predictive system
    

In [ ]:
X_new = X_test[1]
prediction = model.predict(X_new)
print(prediction)


if(prediction[0] == 0):
  print("The news is real")
else:
  print("The news is fake")

[1]
The news is fake


In [ ]:
print(Y_test[1])

1
